<a href="https://colab.research.google.com/github/sathvik8317/geniusevil/blob/main/Fake_News_Prediciton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/data.csv')

In [5]:
df.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


0-> real
1- > fake

In [6]:
df.shape

(4009, 4)

In [7]:
df['Label'].value_counts()

Label
0    2137
1    1872
Name: count, dtype: int64

In [8]:
df.isnull().sum()

URLs         0
Headline     0
Body        21
Label        0
dtype: int64

In [9]:
# replacing the null values with empty string
df = df.fillna('')

In [10]:
df['content'] = df['Headline'] + ' ' + df['Body']

In [73]:
df.head()

,URLs,Headline,Body,Label,content
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,four way bob corker skewer donald trump imag c...
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,linklat war veteran comedi speak modern americ...
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,trump fight corker jeopard legisl agenda feud ...
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,egypt cheiron win tie pemex mexican onshor oil...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,jason aldean open snl vega tribut countri sing...


In [12]:
#seperating the content and the labels

In [13]:
X = df.drop(columns = 'Label',axis = 1)

In [14]:
Y = df['Label']

In [15]:
print(X)

                                                   URLs  \
0     http://www.bbc.com/news/world-us-canada-414191...   
1     https://www.reuters.com/article/us-filmfestiva...   
2     https://www.nytimes.com/2017/10/09/us/politics...   
3     https://www.reuters.com/article/us-mexico-oil-...   
4     http://www.cnn.com/videos/cnnmoney/2017/10/08/...   
...                                                 ...   
4004  http://beforeitsnews.com/sports/2017/09/trends...   
4005  http://beforeitsnews.com/u-s-politics/2017/10/...   
4006  https://www.activistpost.com/2017/09/ron-paul-...   
4007  https://www.reuters.com/article/us-china-pharm...   
4008  http://beforeitsnews.com/u-s-politics/2017/10/...   

                                               Headline  \
0            Four ways Bob Corker skewered Donald Trump   
1     Linklater's war veteran comedy speaks to moder...   
2     Trump’s Fight With Corker Jeopardizes His Legi...   
3     Egypt's Cheiron wins tie-up with Pemex for Mex...

In [16]:
print(Y)

0       1
1       1
2       1
3       1
4       1
       ..
4004    0
4005    0
4006    0
4007    1
4008    0
Name: Label, Length: 4009, dtype: int64


In [17]:
# Stemming

In [18]:
# It is the process of reducing a word into its root word
# actor,actress ,acting into act

In [19]:
port_stem = PorterStemmer()

In [20]:
def stemming(content):
    # Replace non-alphabetic characters with spaces
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)

    # Convert to lowercase and split into words
    stemmed_content = stemmed_content.lower().split()

    # Apply stemming and remove stopwords
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]

    # Join the stemmed words back into a string
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

"The quick brown" as After splitting: ['The', 'quick', 'brown']

After joining: The quick brown

Suppose content initially contains a string like "Hello! How are you? I'm doing well.". After applying re.sub('[^a-zA-Z]', ' ', content), the stemmed_content would be "Hello How are you I m doing well ", where punctuation and other non-alphabetical characters have been replaced by spaces.

The purpose of this line of code is to remove or replace all non-alphabetical characters (characters that are not letters) from the content string with spaces.

This type of transformation is commonly used as a preprocessing step in natural language processing (NLP) tasks, especially when dealing with text data.

In [21]:
df['content'] = df['content'].apply(stemming)

In [22]:
print(df['content'])

0       four way bob corker skewer donald trump imag c...
1       linklat war veteran comedi speak modern americ...
2       trump fight corker jeopard legisl agenda feud ...
3       egypt cheiron win tie pemex mexican onshor oil...
4       jason aldean open snl vega tribut countri sing...
                              ...                        
4004    trend watch trend watch reader think stori fac...
4005    trump jr soon give minut speech trump jr soon ...
4006                       ron paul trump anarch altright
4007    china accept oversea trial data bid speed drug...
4008    vice presid mike penc leav nfl game anti ameri...
Name: content, Length: 4009, dtype: object


In [53]:
X = df['content'].values
Y = df['Label'].values

In [54]:
print(X)

['four way bob corker skewer donald trump imag copyright getti imag sunday morn donald trump went twitter tirad member parti exactli huge news far first time presid turn rhetor cannon rank time howev attack particularli bite person essenti call tennesse senat bob corker chair power senat foreign relat committe coward run elect said mr corker beg presid endors refus give wrongli claim mr corker support iranian nuclear agreement polit accomplish unlik colleagu mr corker free worri immedi polit futur hold tongu skip twitter post senbobcork shame white hous becom adult day care center someon obvious miss shift morn senat bob corker senbobcork octob report end though spoke new york time realli let presid four choic quot tennesse senat interview time particularli damn know presid tweet thing true know everyon know realli sugarcoat one mr corker flat say presid liar everyon know senat particular challeng mr trump insist unsuccess plead endors accus much broader mr corker presid use someth aki

In [25]:
print(Y)

[1 1 1 ... 0 1 0]


In [55]:
X.shape

(4009,)

In [56]:
Y.shape

(4009,)

In [60]:
# converting textual data into numerical data
vectorizer =  TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [61]:
print(X.shape)

(4009, 30459)


In [62]:
print(Y.shape)

(4009,)


Splitting the data into training and test data

In [63]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2,stratify=Y)

In [64]:
model = LogisticRegression()

In [65]:
model.fit(X_train,Y_train)

LogisticRegression()

In [66]:
X_train_pred = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_pred,Y_train)
print(training_data_accuracy)

0.990645463049579


X_train: This represents the input features (independent variables) of the training dataset. Each row corresponds to a set of features (e.g., characteristics of an object or person).

X_train_pred: This variable holds the predicted target values (predictions) made by the model based on the input features in X_train.

In [67]:
X_test_pred = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_pred,Y_test)
print(testing_data_accuracy)

0.9613466334164589


Training Data Accuracy Greater Than Testing Data Accuracy:
This scenario suggests that the model may be overfitting the training data. Overfitting occurs when the model learns not only the underlying patterns in the training data but also noise and random fluctuations specific to the training dataset.

Training Data Accuracy Lesser Than Testing Data Accuracy:
This situation is less common but can occur if the model is underfitting the training data. Underfitting happens when the model is too simplistic or not trained enough to capture the underlying patterns in the training data. In such cases, the model may perform poorly on both the training and testing datasets, but its accuracy on testing data could still be higher due to chance or other factors.

Training Data Accuracy Equal to Testing Data Accuracy:
Ideally, a model's performance should be similar on both training and testing datasets, indicating that it generalizes well to new, unseen data. If the accuracies are similar and high, it suggests that the model has learned the underlying patterns effectively without overfitting or underfitting.

Making a Predictive System

In [68]:
X_new = X_test[100]

prediction = model.predict(X_new)
print(prediction)
if(prediction ==0):
  print("The news is real")
else:
  print("The news is fake")

[0]
The news is real


In [70]:
print(Y_test[100])

0


In [51]:
print(Y_test[0])

0


Integration with Flask API or GUI interface is left-over

In [50]:
print(Y_test[0])

0


In [72]:
def predict_news_category(news_text):
    # Preprocess the input news text
    text_vector = vectorizer.transform([news_text])

    # Make prediction using the trained model
    prediction = model.predict(text_vector)

    if prediction == 0:
        return "The news is real"
    else:
        return "The news is fake"

# Continuous prediction loop
while True:
    # Get news input from user
    input_news = input("Enter news article (type 'exit' to quit): ")

    if input_news.lower() == 'exit':
        break

    # Perform prediction
    result = predict_news_category(input_news)

    # Display prediction result
    print(result)


Enter news article (type 'exit' to quit): Today it will rain
The news is fake
Enter news article (type 'exit' to quit): THere is a dog
The news is real
Enter news article (type 'exit' to quit): Four ways Bob Corker skewered Donald Trump Image copyright Getty Images On Sunday morning, Donald Trump went off on a Twitter tirade against a member of his own party. This, in itself, isn't exactly huge news. It's far from the first time the president has turned his rhetorical cannons on his own ranks. This time, however, his attacks were particularly biting and personal. He essentially called Tennessee Senator Bob Corker, the chair of the powerful Senate Foreign Relations Committee, a coward for not running for re-election. He said Mr Corker "begged" for the president's endorsement, which he refused to give. He wrongly claimed that Mr Corker's support of the Iranian nuclear agreement was his only political accomplishment. Unlike some of his colleagues, Mr Corker - free from having to worry abo